In [1]:
import sys
sys.path.append('EpiGePT')

In [3]:
import torch
torch.cuda.is_available()

/project/deeprna_data/epigept/epigept-env/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
import os
#from pyfasta import Fasta
import numpy as np
import pandas as pd
#import model_hg38.EpiGePT as epigept
#from model_hg38.config import *
#from model_hg38.utils import *

In [11]:
model = epigept.EpiGePT(WORD_NUM,TF_DIM,BATCH_SIZE)
model = load_weights(model,'/project/deeprna_data/epigept/model.ckpt')

In [14]:
SEQ_LENGTH = 128000
input_tf_feature = np.random.rand(1000, 711) # 711 TFs
input_seq_feature = np.zeros((1,4,SEQ_LENGTH))
predict = model_predict(model,input_seq_feature,input_tf_feature)
predict.shape # (BATCH_SIZE, Number of bins, Number of epigenomic profiles)

(1, 1000, 8)

In [38]:
ref_tf_expression = pd.read_csv('tf_expression.csv',header=0,sep='\t',index_col=[0])

def quantile_norm(df,ref_exp):
    """quantile normalization
    Arg:
        df: Pandas DataFrame with TFs x cells
    Return:
        normalized Pandas DataFrame 
    """
    rank_mean = ref_exp.stack().groupby(ref_exp.rank(method='first').stack().astype(int)).mean()
    return df.rank(method='min').stack().astype(int).map(rank_mean).unstack()

cell_id = 0 # index for the cell type
N = 64 # number for cell types
tf_TPM_expression = pd.read_csv('EXAMPLE_TPM.csv',header=0,sep='\t',index_col=[0]) # TPM value of 711 TFs
tf_expression = quantile_norm(tf_TPM_expression,ref_tf_expression)
tf_expression = np.array(tf_expression.iloc[:,cell_id]) # quntile transformed TPM values

In [42]:
ref_tf_expression

,1,2,3,4,5,6,7,8,9,10,...,96,97,98,99,100,101,102,103,104,105
AFP,1.577789,1.137986,3.262636,1.541115,2.096914,0.941996,78.170130,1.430745,1.668556,2.380495,...,16.898590,0.995006,0.000000,0.000000,0.949619,1.562175,0.000000,0.749428,0.731091,0.000000
AHR,11.625045,5.797212,9.114685,1.167180,8.728365,4.393192,15.765104,115.001310,7.401119,19.286982,...,13.277387,29.340546,89.869160,12.563602,15.644994,43.157825,38.817290,70.499200,32.967530,41.972320
AIRE,3.859208,0.924033,0.000000,0.000000,1.780244,1.256665,0.996202,0.995006,2.096914,0.000000,...,8.513892,1.175513,1.070703,0.000000,1.058877,0.870245,0.000000,0.933564,0.992850,1.148756
ALX1,0.606442,4.925012,0.000000,0.000000,1.266805,0.941996,0.000000,2.960496,1.006817,1.438789,...,0.834547,1.061191,0.000000,0.000000,0.858172,1.148756,0.000000,1.252141,1.743177,1.676488
ALX3,0.731740,1.773563,0.000000,0.000000,5.921328,1.562175,0.000000,0.000000,0.801987,0.000000,...,1.058877,0.916141,0.000000,0.000000,15.214487,1.009232,0.000000,1.124038,1.011476,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF711,4.882535,8.900022,7.401119,10.664214,2.108742,8.196231,6.101785,4.202334,7.722016,1.942604,...,17.373130,4.069373,1.860127,1.743177,4.168037,1.646783,1.849655,7.674312,10.922078,1.710760
ZNF740,24.498137,18.834179,13.607840,17.953228,12.173142,12.018184,16.008425,11.692537,16.008425,4.427230,...,19.908827,9.186198,12.407563,6.829425,5.552056,11.466254,14.073768,7.197051,11.054543,5.034946
ZNF8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ZSCAN22,3.914630,2.941975,4.393192,6.062632,3.821951,4.459918,3.347780,3.510115,4.734438,3.172305,...,2.380495,2.760720,2.278904,2.558273,2.123426,2.188707,2.524062,1.876192,2.188707,2.148822


In [69]:
tf_expression_ours = np.load('/project/deeprna_data/pretraining_data_final2/tf_expression.npy')
tf_expression_ours = ((tf_expression_ours - 1) ** 10) - 0.1
tf_expression_ours.shape

(600, 2891)

In [47]:
with open('/project/deeprna/data/tf_genes/trans_regulators_final_hgnc.txt', 'r') as f:
    tf_list = f.read().strip().split()

In [51]:
tf_intersection = {x:i for i,x in enumerate(tf_list) if x in ref_tf_expression.index}
len(tf_intersection)

695

In [70]:
tissue_39_df = pd.DataFrame(index=ref_tf_expression.index, columns=['tpm'])
tissue_39_df.loc[list(tf_intersection), 'tpm'] = tf_expression_ours[39, list(tf_intersection.values())]
tissue_39_df = tissue_39_df.fillna(0)
tissue_39_df

,tpm
AFP,0.000000
AHR,13.313547
AIRE,0.555251
ALX1,0.900000
ALX3,0.900000
...,...
ZNF711,-0.099927
ZNF740,0.574740
ZNF8,-0.097622
ZSCAN22,-0.100000


In [71]:
tissue_39_qn = quantile_norm(tissue_39_df,ref_tf_expression)
tissue_39_qn

,tpm
AFP,0.983197
AHR,19.615995
AIRE,1.673011
ALX1,2.458637
ALX3,2.458637
...,...
ZNF711,0.264297
ZNF740,2.306272
ZNF8,0.425890
ZSCAN22,0.015500


In [73]:
tissue_39_qn.to_csv('tissue_39.csv')

In [52]:
tf_intersection

{'AHR': 28,
 'AIRE': 30,
 'ALX1': 34,
 'ALX3': 35,
 'ALX4': 36,
 'AR': 50,
 'ARID3A': 59,
 'ARID3B': 60,
 'ARID5A': 62,
 'ARID5B': 63,
 'ARNT': 65,
 'ARNT2': 66,
 'ARX': 67,
 'ASCL1': 72,
 'ASCL2': 73,
 'ATF1': 79,
 'ATF2': 80,
 'ATF3': 81,
 'ATF4': 82,
 'ATF5': 83,
 'ATF6': 84,
 'ATF6B': 85,
 'ATF7': 86,
 'ATOH1': 89,
 'BACH1': 96,
 'BACH2': 97,
 'BARHL1': 101,
 'BARHL2': 102,
 'BARX1': 103,
 'BARX2': 104,
 'BATF': 105,
 'BATF3': 106,
 'BBX': 108,
 'BCL6': 112,
 'BCL6B': 113,
 'BDP1': 118,
 'BHLHA15': 119,
 'BHLHE22': 120,
 'BHLHE23': 121,
 'BHLHE40': 122,
 'BHLHE41': 123,
 'BPTF': 133,
 'BRCA1': 134,
 'BSX': 143,
 'CCNT2': 176,
 'CDC5L': 179,
 'CDX1': 182,
 'CDX2': 183,
 'CEBPA': 188,
 'CEBPB': 189,
 'CEBPD': 190,
 'CEBPE': 191,
 'CEBPG': 192,
 'CENPB': 201,
 'CHD2': 214,
 'CLOCK': 235,
 'CPEB1': 244,
 'CREB1': 255,
 'CREB3': 256,
 'CREB3L1': 257,
 'CREB3L2': 258,
 'CREB3L4': 259,
 'CREB5': 260,
 'CREM': 264,
 'CRX': 268,
 'CTCF': 280,
 'CTCFL': 281,
 'CUX1': 285,
 'CUX2': 286,
 'DBP

## Bed file for epigept input

In [22]:
def parse_bed_file_with_coords(bed_path):
    """
    Parse a BED file and return a list of tuples (chr, start, end).
    Assumes no header and at least three columns.
    """
    df = pd.read_csv(bed_path, sep='\t', header=None, names=["chr", "start", "end", "fold"])
    coords = []
    for _, row in df.iterrows():
        coords.append((row["chr"], int(row["start"]), int(row["end"])))
    return coords

def tile_region(chrom, start, end, sequence_length, stride, drop_last=True):
    """
    Given a region defined by (chrom, start, end),
    create tiles of length sequence_length with the given stride.
    Returns a list of tuples: (chrom, tile_start, tile_end)
    """
    tiles = []
    region_length = end - start
    if region_length < sequence_length:
        return tiles
    current_start = start
    while current_start + sequence_length <= end:
        tiles.append((chrom, str(current_start), str(current_start + sequence_length)))
        current_start += stride
    # Optionally, if drop_last is False, add a shifted tile.
    return tiles

def tile_regions(regions, sequence_length, stride, drop_last=True):
    """
    For a list of regions (each as (chr, start, end)), return all tiles.
    """
    tiled = []
    for region in regions:
        chrom, start, end = region
        tiled.extend(tile_region(chrom, start, end, sequence_length, stride, drop_last))
    return tiled

In [23]:
bed = parse_bed_file_with_coords('/project/deeprna/benchmark/fold3_notf_merged.bed')

tiles = tile_regions(bed, 128000, 128000)

len(tiles)

2224

In [24]:
with open('/project/deeprna_data/epigept/fold3_notf_epigept.bed', 'w') as f:
    f.write('\n'.join(['\t'.join(x) for x in tiles]))